In [1]:
import warnings
import pandas as pd
import numpy as np
from scipy import stats
import sklearn as sk
import itertools
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
import datetime
import re
import praw
import jsonlines


[nltk_data] Downloading package punkt to /home/ruru22/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ruru22/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ruru22/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
client_id = 'sIXZTihLNiKiHw'
client_secret = 'EjfAsmz5z8mDbZohe4UPYTPIZsYmOQ'
user_agent = 'Test1'

date_limits = ['2019-03-01 00:00:00', 
               '2019-06-01 00:00:00', 
               '2019-09-01 00:00:00', 
               '2020-01-01 00:00:00', 
               '2020-03-01 00:00:00', 
               '2020-06-01 00:00:00', 
               '2020-09-01 00:00:00', 
               '2021-01-01 00:00:00', 
               '2021-03-01 00:00:00', 
               '2021-06-01 00:00:00'
            ]

COUNT = len(date_limits)

In [5]:
def dfFromCSV(filename):
    posts = pd.read_csv(filename)
    return posts

def dfToCSV(posts, filename):
    posts.to_csv(filename, index=False)

def postsFromNDJSON(filename):
    posts = []
    with jsonlines.open(filename) as reader:
        for line in reader:
            line = line["Item"]
            posts.append([line["created"]["S"], line["title"]["S"], line["id"]["S"]])
    posts = pd.DataFrame(posts,columns=['created', 'title', 'id'])

    return posts

def sortByTime(data):
    data = data.sort_values(by='created')
    return data

posts = postsFromNDJSON("worldnews.jsonl")
posts = sortByTime(posts)
posts.drop(posts.loc[posts['title'] == 'None'].index, inplace=True)
dfToCSV(posts=posts, filename="worldnews.csv")
print(posts)


                    created  \
360671  2018-12-31 02:25:14   
270512  2018-12-31 02:27:21   
112640  2018-12-31 02:27:56   
294115  2018-12-31 02:29:05   
214742  2018-12-31 02:29:18   
...                     ...   
498320  2021-04-09 21:26:18   
427024  2021-04-09 21:26:19   
816977  2021-04-09 21:26:20   
144817  2021-04-09 21:26:21   
516521  2021-04-09 21:26:28   

                                                    title      id  
360671  پسکوف: هیچ اقدامی علیه مظنونان معرفی گردیده در...  ab2og2  
270512  One of the companies contracted by the governm...  ab2p2z  
112640  Donald Trump: Outgoing chief of staff John Kel...  ab2p8u  
294115  Todd Bowles Fired as Jets Head Coach After 4 S...  ab2pl2  
214742  Macron's former aide admits using diplomatic p...  ab2pn3  
...                                                   ...     ...  
498320  Prince Charles Visits Queen Elizabeth II After...  mnr2wp  
427024  Job postings hint at winners of NYC and London...  mnr2x1  
816977  Tell Us

In [132]:

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess(text):
    lemmatizer = WordNetLemmatizer()

    # Tokenize words
    tokens = word_tokenize(text)

    # remove links
    words = []
    for token in tokens: 
        if 'http' not in token and 'www' not in token:
            words.append(token)
            #print(token)

    # Remove stopwords and lemmatize words
    words = [lemmatizer.lemmatize(word.lower()) for word in words if word.isalnum() and word.lower() not in stop_words]
    #print(words)

    words_split = []
    for word in words:
        if isEnglish(word):
            words_split.extend((re.split('(\d+)',word)))

    return ' '.join(words_split)

def isEnglish(text):
    try:
        text.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True


def separateNumbersAlphabets(str):
    numbers = re.findall(r'[0-9]+', str)
    alphabets = re.findall(r'[a-zA-Z]+', str)
    print(*numbers)
    print(*alphabets)

posts = dfFromCSV('worldnews.csv')

start = datetime.datetime.now()

lemmed = []

for idx in posts.index:
    if idx % 50000 == 0:
        print(idx / 1000)
    text = posts['title'][idx]

    text = preprocess(text)

    if isEnglish(text):
        lemmed.append(text)
    else:
        lemmed.append('DELETEME')

stop = datetime.datetime.now()
print("Took", stop - start, "seconds")

posts["title"] = lemmed

posts.drop(posts.loc[posts['title'] == 'DELETEME'].index, inplace=True)
posts.drop(posts.loc[posts['title'] == ''].index, inplace=True)

#print(posts)
dfToCSV(posts, 'lemmatized.csv')

0.0


KeyboardInterrupt: 

In [159]:
def downloadPosts(ids):
    reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent, check_for_async=False)
    posts_raw = []

    for idx in range(0,len(ids)):
        ids[idx] = "t3_" + ids[idx]

    info = reddit.info(fullnames=ids)

    texts = []
    num_comments = []
    scores = []
    authors = []
    titles = []
    idxs = []
    used_ids = []
    times = []

    post_data = pd.DataFrame(list(zip([], titles, authors, scores, num_comments, texts)),columns=['id', 'title', 'author', 'score', 'num_comments', 'text'])
    print(post_data)

    for post in info:
        if post.score < 100:
            continue

        text = ''
        for top_level_comment in post.comments:
            if(hasattr(top_level_comment, 'body')):
                processed = preprocess(top_level_comment.body)
                if processed is not None:
                    text += processed + ' '
        texts.append(text)
        num_comments.append(len(post.comments))
        scores.append(post.score)
        authors.append(post.author)
        titles.append(preprocess(post.title))
        used_ids.append(post.id)
        times.append(post.created_utc)

    post_data = pd.DataFrame(list(zip(used_ids, times, titles, authors, scores, num_comments, texts)),columns=['id', 'time', 'title', 'author', 'score', 'num_comments', 'text'])

    return post_data



#reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent, check_for_async=False)
#submission = reddit.submission('16dneuo')
#print(len(submission.comments))
#for top_level_comment in submission.comments:
#    print(top_level_comment.body)
def divide_chunks(l, n):
    # looping till length l
    for i in range(0, len(l), n): 
        yield l[i:i + n]

worldnews = dfFromCSV("lemmatized_ascii.csv")
for rem in range(0,1000):
    reduced = worldnews.iloc[[*range(rem,len(worldnews),1000)], :].copy()
    reduced = reduced['id'].to_list()
    chunks = list(divide_chunks(reduced, 1000))
    print(len(chunks), len(chunks[0]), len(chunks[-1]))
    for idx,chunk in enumerate(chunks):
        dfToCSV(downloadPosts(chunk), "comments_" + str(rem) + "_" + str(idx) +".csv")
  
#print(len(reduced))
#dfToCSV(downloadPosts(reduced['id'].to_list()), "comments.csv")
#dfToCSV(downloadPosts(['kljtg8']), "comments.csv")



1 790 790
Empty DataFrame
Columns: [id, title, author, score, num_comments, text]
Index: []
1 790 790
Empty DataFrame
Columns: [id, title, author, score, num_comments, text]
Index: []
1 790 790
Empty DataFrame
Columns: [id, title, author, score, num_comments, text]
Index: []
1 790 790
Empty DataFrame
Columns: [id, title, author, score, num_comments, text]
Index: []
1 790 790
Empty DataFrame
Columns: [id, title, author, score, num_comments, text]
Index: []
1 790 790
Empty DataFrame
Columns: [id, title, author, score, num_comments, text]
Index: []
1 790 790
Empty DataFrame
Columns: [id, title, author, score, num_comments, text]
Index: []
1 790 790
Empty DataFrame
Columns: [id, title, author, score, num_comments, text]
Index: []
1 790 790
Empty DataFrame
Columns: [id, title, author, score, num_comments, text]
Index: []
1 790 790
Empty DataFrame
Columns: [id, title, author, score, num_comments, text]
Index: []
1 790 790
Empty DataFrame
Columns: [id, title, author, score, num_comments, text

KeyboardInterrupt: 

In [233]:
temps = []
for i in range(0,167):
    temps.append(dfFromCSV('comments_' + str(i) + "_0.csv"))

df = pd.concat(temps)
start = df['time'].min()
df['time'] = df['time'] - start
df = df.sort_values(by='time')
df['month'] = df['time'] / 2592000
df['month'] = (df['month'] - 0.5).round()
df = df.reset_index()

dfToCSV(df, '_sorted.csv')
df

,index,id,time,title,author,score,num_comments,text,month
0,0,ab2pn3,0.0,macron former aide admits using diplomatic pas...,NaN,301,5,pretty huge story france let give detail case ...,-0.0
1,0,ab33i1,2801.0,bacteria found ancient irish soil halt growth ...,bhel_,5058,62,something muck dirt buddy mine gross wart hand...,-0.0
2,0,ab36ix,3411.0,korea ban use disposable shopping bag supermarket,edwinksl,8930,81,cafe illegal serve drink paper plastic cup dri...,-0.0
3,0,ab3esx,5137.0,footage show saudi hit team carrying bag belie...,AWACS-Thunderhead,2022,24,anyone know turkey managed get audio recording...,-0.0
4,0,ab3eyp,5167.0,trump worst perpetrator false information amer...,glasier,72507,35,remember said releasing jfk file still waiting...,-0.0
...,...,...,...,...,...,...,...,...,...
8686,42,mndr89,71738407.0,pressure xinjiang china take aim overseas uigh...,edourdoo1,180,8,friday event official took aim database set ov...,27.0
8687,53,mne5su,71740259.0,norwegian prime minister fined police breaking...,NaN,2634,26,deleted first thought norwegian good able dodg...,27.0
8688,51,mne6ah,71740331.0,south korea launch first homegrown supersonic ...,brownandbumpy,178,9,final test completed future south korea become...,27.0
8689,63,mnekkn,71742080.0,bike lane canada commits 400 m build sustaina...,ellalingling,106,9,putting infrastructure get 8 month long summ...,27.0
